In [1]:
import json
import os
from tabulate import tabulate
import pandas as pd


In [14]:
#1

def tworz_tabele_dla_wszystkich_sekcji(json_data):
    wszystkie_sekcje = ['uczniowie', 'chlopcy', 'dziewczeta']
    wynik = {}

    for sekcja in wszystkie_sekcje:
        top_rows = []
        min_rows = []
        dane_sekcji = json_data.get(sekcja, {})

        for zawod, wartosci in dane_sekcji.get("top", []):
            top_rows.append({
                "zawód": zawod,
                "uczniowie": wartosci["uczniowie"],
                "chłopcy": wartosci["chlopcy"],
                "dziewczęta": wartosci["dziewczeta"]
            })

        for zawod, wartosci in dane_sekcji.get("min", []):
            min_rows.append({
                "zawód": zawod,
                "uczniowie": wartosci["uczniowie"],
                "chłopcy": wartosci["chlopcy"],
                "dziewczęta": wartosci["dziewczeta"]
            })

        wynik[sekcja] = {
            "top": pd.DataFrame(top_rows),
            "min": pd.DataFrame(min_rows)
        }

    return wynik

In [17]:
#1
def zwizualizuj_wojewodztwa_zawody(json_path, sciezka_rok_json):
    # Wyodrębnij rok z nazwy pliku (np. "WojewodztwaZawody_20232024_report.json")
    nazwa_pliku = os.path.basename(json_path)
    rok = os.path.basename(json_path)[-20:-12]
    
    # Wczytaj dane z pliku z zawodami
    with open(json_path, "r", encoding="utf-8") as plik:
        dane_z_pliku = json.load(plik)

    # Wczytaj dane z pliku z całkowitą liczbą uczniów
    with open(sciezka_rok_json, "r", encoding="utf-8") as plik_rok:
        dane_rok = json.load(plik_rok)

    calkowita_liczba_uczniow = dane_rok.get("rok", None)
    if calkowita_liczba_uczniow is None:
        print(f"Nie znaleziono całkowitej liczby uczniów w pliku {sciezka_rok_json} pod kluczem 'rok'.")
        return

    # Tworzenie tabel
    tabele = tworz_tabele_dla_wszystkich_sekcji(dane_z_pliku)

    print(f"\n==================== ROK {rok} ====================")
    
    for sekcja, dane in tabele.items():
        df_top = dane['top'].copy()
        df_min = dane['min'].copy()

        # Dodaj kolumnę z rokiem
        df_top["rok"] = rok
        df_min["rok"] = rok

        # Dodaj kolumny procentowe (względem całkowitej liczby uczniów)
        df_top["% uczniów"] = (df_top["uczniowie"] / calkowita_liczba_uczniow * 100).round(2)
        df_min["% uczniów"] = (df_min["uczniowie"] / calkowita_liczba_uczniow * 100).round(2)

        print(f"\n▶ {sekcja.upper()} – TOP")
        print(tabulate(df_top, headers="keys", tablefmt="grid", showindex=False))

        print(f"\n▼ {sekcja.upper()} – MIN")
        print(tabulate(df_min, headers="keys", tablefmt="grid", showindex=False))

        print("\n" + "-"*70)
        
zwizualizuj_wojewodztwa_zawody("20222023/Uczniowie_w_Zawodach_20222023_report.json", "20222023/Rok_LiczbaUczniow_20222023.json")
zwizualizuj_wojewodztwa_zawody("20232024/Uczniowie_w_Zawodach_20232024_report.json", "20232024/Rok_LiczbaUczniow_20232024.json")
zwizualizuj_wojewodztwa_zawody("20242025/Uczniowie_w_Zawodach_20242025_report.json", "20242025/Rok_LiczbaUczniow_20242025.json")


==================== ROK 20222023 ====================

▶ UCZNIOWIE – TOP
+-------------------------------------------+-------------+-----------+--------------+----------+-------------+
| zawód                                     |   uczniowie |   chłopcy |   dziewczęta |      rok |   % uczniów |
+===========================================+=============+===========+==============+==========+=============+
| technik informatyk                        |      106494 |    106494 |            0 | 20222023 |        8.58 |
+-------------------------------------------+-------------+-----------+--------------+----------+-------------+
| technik logistyk                          |       64731 |     64731 |            0 | 20222023 |        5.22 |
+-------------------------------------------+-------------+-----------+--------------+----------+-------------+
| przygotowanie do kształcenia zawodowego   |       63603 |     63603 |            0 | 20222023 |        5.13 |
+----------------------------

In [8]:
import pandas as pd

def tworz_tabele_typ_szkola(dane_json):
    wyniki = {}

    for typ in ['top', 'min']:
        lista = dane_json.get(typ, [])
        dane = []

        for (wojewodztwo, typ_szkoly), liczba_uczniow in lista:
            dane.append({
                'województwo': wojewodztwo,
                'typ szkoły': typ_szkoly,
                'liczba uczniów': liczba_uczniow
            })

        df = pd.DataFrame(dane)
        wyniki[typ] = df

    return wyniki


In [26]:
#4
def zwizualizuj_szkoly_typPodmiotu_wojewodztwa(json_path):
    rok = os.path.basename(json_path)[-20:-12]

    with open(json_path, "r", encoding="utf-8") as plik:
        dane_z_pliku = json.load(plik)
    
    tabelki = tworz_tabele_typ_szkola(dane_z_pliku)

    tabelki['top']["rok"] = rok
    tabelki['min']["rok"] = rok

    print(f"\n=== ROK {rok} ===")

    print("\nTOP:")
    print(tabulate(tabelki['top'], headers='keys', tablefmt='grid', showindex=False))

    print("\nMIN:")
    print(tabulate(tabelki['min'], headers='keys', tablefmt='grid', showindex=False))

zwizualizuj_szkoly_typPodmiotu_wojewodztwa("20222023/Szkoly_TypPodmiotu_Wojewodztwa_20222023_report.json")
zwizualizuj_szkoly_typPodmiotu_wojewodztwa("20232024/Szkoly_TypPodmiotu_Wojewodztwa_20232024_report.json")
zwizualizuj_szkoly_typPodmiotu_wojewodztwa("20242025/Szkoly_TypPodmiotu_Wojewodztwa_20242025_report.json")



=== ROK 20222023 ===

TOP:
+---------------+---------------------------+------------------+----------+
| województwo   | typ szkoły                |   liczba uczniów |      rok |
+===============+===========================+==================+==========+
| wielkopolskie | branżowa szkoła i stopnia |             3891 | 20222023 |
+---------------+---------------------------+------------------+----------+
| śląskie       | technikum                 |             3310 | 20222023 |
+---------------+---------------------------+------------------+----------+
| mazowieckie   | technikum                 |             2987 | 20222023 |
+---------------+---------------------------+------------------+----------+
| małopolskie   | branżowa szkoła i stopnia |             2797 | 20222023 |
+---------------+---------------------------+------------------+----------+
| wielkopolskie | technikum                 |             2634 | 20222023 |
+---------------+---------------------------+---------------